In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy import stats
from scipy.stats import linregress
import cpi
cpi.update()

In [2]:
# CSV Path
imdb_movies_csv = "Data/imdb_movies.csv"
box_office_csv = "Data/boxoffice.csv"

In [3]:
# Variables for the seperate data frames 
imdb_df = pd.read_csv(imdb_movies_csv)
box_office_df = pd.read_csv(box_office_csv)

In [4]:
# Removing columns I do not need
col_imdb_df = imdb_df[['imdb_title_id', 'original_title', 'year', 'date_published', 'genre', 'duration', 'country', 
                         'language', 'director', 'writer', 'production_company', 'actors', 'avg_vote', 'votes']]

col_box_office_df = box_office_df[['title', 'lifetime_gross', 'year']]


In [5]:
# Renaming columns to be more uniform
col_imdb_df = col_imdb_df.rename(columns={'imdb_title_id': "IMDB ID", 'original_title': 'Title', 'year': 'Year', 
                                          'date_published': 'Date Published', 'genre': 'Genre', 'duration': 'Duration',
                                         'country': 'Country', 'language': 'Language', 'director': 'Director', 'writer': 'Writer',
                                         'production_company': 'Production Company', 'actors': 'Actors', 'avg_vote': 'Avg Rating',
                                          'votes': '# of Ratings'})

col_box_office_df = col_box_office_df.rename(columns={'title': 'Title', 'lifetime_gross': 'Lifetime_Gross', 'year': 'Year'})

In [6]:
# Merging data frames
movie_data = pd.merge(col_imdb_df, col_box_office_df, on=['Title', 'Year'], how='inner')

# Splicing the Genre column into multiple rows


# Dropping data that is not available through CPI
movie_data.drop(movie_data[movie_data['Year'] == 1912 ].index , inplace=True)
movie_data.drop(movie_data[movie_data['Year'] == 2019 ].index , inplace=True)
movie_data.drop(movie_data[movie_data['Year'] == 2020 ].index , inplace=True)

In [7]:
# Sort IMDB ID
movie_data.sort_values("IMDB ID", inplace = True)

# Drop duplicate IMDB ID
movie_data.drop_duplicates(subset ="IMDB ID", keep = False, inplace = True)

In [8]:
# Calculating inflation through CPI library 
# Source: https://medium.com/analytics-vidhya/adjusting-for-inflation-when-analysing-historical-data-with-python-9d69a8dcbc27
def inflate_column(data, column):
    
    return data.apply(lambda x: cpi.inflate(x[column], x.Year), axis=1)          
    
movie_data['Lifetime Gross - Adjusted ($)'] = inflate_column(movie_data, 'Lifetime_Gross')

In [9]:
# Converting scientific notation
movie_data["Lifetime Gross - Adjusted ($)"].describe().apply(lambda x: format(x, 'f'))
pd.options.display.float_format = '{:,.2f}'.format

# Formatting columns
movie_data['Lifetime_Gross'] = movie_data['Lifetime_Gross'].astype(float).map("${:,.2f}".format)
movie_data['Lifetime Gross - Adjusted ($)'] = movie_data['Lifetime Gross - Adjusted ($)'].astype(float).map("${:,.2f}".format)

movie_data

,IMDB ID,Title,Year,Date Published,Genre,Duration,Country,Language,Director,Writer,Production Company,Actors,Avg Rating,# of Ratings,Lifetime_Gross,Lifetime Gross - Adjusted ($)
0,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,"Drama, Romance, War",150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",7.20,2935,"$9,183,673.00","$131,165,937.89"
1,tt0017136,Metropolis,1927,1927-02-06,"Drama, Sci-Fi",153,Germany,German,Fritz Lang,"Thea von Harbou, Thea von Harbou",Universum Film (UFA),"Alfred Abel, Gustav Fröhlich, Rudolf Klein-Rog...",8.30,148396,"$1,236,166.00","$18,162,901.79"
2,tt0017354,The Sea Beast,1926,1926-01-15,"Action, Adventure, Fantasy",136,USA,English,Millard Webb,"Herman Melville, Bess Meredyth",Warner Bros.,"John Barrymore, Dolores Costello, George O'Har...",6.50,126,"$814,000.00","$11,757,333.22"
3,tt0021749,City Lights,1931,1931-08-21,"Comedy, Drama, Romance",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Virginia Cherrill, Florence Lee, Harry Myers, ...",8.50,152716,"$19,181.00","$322,615.59"
4,tt0027977,Modern Times,1936,1936-10-16,"Comedy, Drama, Family",87,USA,English,Charles Chaplin,Charles Chaplin,Charles Chaplin Productions,"Charles Chaplin, Paulette Goddard, Henry Bergm...",8.50,197969,"$163,577.00","$3,008,604.68"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7729,tt8235296,The Trump Prophecy,2018,2018-10-02,Drama,120,USA,English,Stephan Schultze,"Mary Colbert, Rick Eldridge",ReelWorks Studios,"Chris Nelson, Paulette Todd, Karen Boles, Don ...",3.00,1669,"$671,198.00","$683,359.95"
7730,tt8264546,1991,2018,2018-07-27,Comedy,141,Canada,"French, Italian, English, German, Greek, Spanish",Ricardo Trogi,Ricardo Trogi,NaN,"Sandrine Bisson, Jean-Carl Boucher, Mamoudou C...",7.10,501,"$2,342,264.00","$2,384,705.27"
7735,tt8427036,Helicopter Eela,2018,2018-10-12,Drama,135,India,Hindi,Pradeep Sarkar,"Mitesh Shah, Anand Gandhi",Ajay Devgn Ffilms,"Kajol, Riddhi Sen, Tota Roy Chowdhury, Neha Dh...",5.50,800,"$71,971.00","$73,275.10"
7739,tt8581198,Jane and Emma,2018,2018-10-12,Drama,90,USA,English,Chantelle Squires,Melissa Leilani Larson,NaN,"Danielle Deadwyler, Emily Goss, Brad Schmidt, ...",5.90,115,"$380,030.00","$386,916.05"


In [11]:
# Creating a stacked df, exploding the Genre column into separate rows
# Source: https://www.semicolonworld.com/question/43032/split-explode-pandas-dataframe-string-entry-to-separate-rows
stacked_movie_data = (movie_data.set_index(movie_data.columns.drop('Genre',1).tolist())
                    .Genre.str.split(',', expand=True)
                    .stack()
                    .reset_index()
                    .rename(columns={0:'Genre'})
                     .loc[:, movie_data.columns])

# Creating a stacked df, exploding the Genre column into separate rows
# Source: https://www.semicolonworld.com/question/43032/split-explode-pandas-dataframe-string-entry-to-separate-rows
stacked_movie_data_actors = (stacked_movie_data.set_index(stacked_movie_data.columns.drop('Actors',1).tolist())
                    .Actors.str.split(',', expand=True)
                    .stack()
                    .reset_index()
                    .rename(columns={0:'Actors'})
                     .loc[:, stacked_movie_data.columns])

In [12]:
stacked_movie_data_actors

,IMDB ID,Title,Year,Date Published,Genre,Duration,Country,Language,Director,Writer,Production Company,Actors,Avg Rating,# of Ratings,Lifetime_Gross,Lifetime Gross - Adjusted ($)
0,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,Drama,150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,Pomeroy Cannon,7.20,2935,"$9,183,673.00","$131,165,937.89"
1,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,Drama,150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,Josef Swickard,7.20,2935,"$9,183,673.00","$131,165,937.89"
2,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,Drama,150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,Bridgetta Clark,7.20,2935,"$9,183,673.00","$131,165,937.89"
3,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,Drama,150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,Rudolph Valentino,7.20,2935,"$9,183,673.00","$131,165,937.89"
4,tt0012190,The Four Horsemen of the Apocalypse,1921,1923-01-08,Drama,150,USA,None,Rex Ingram,"Vicente Blasco Ibáñez, June Mathis",Metro Pictures Corporation,Virginia Warwick,7.20,2935,"$9,183,673.00","$131,165,937.89"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
277451,tt8925540,Racetime,2018,2018-12-14,Comedy,89,Canada,English,"Benoît Godbout, Jean-François Pouliot","Claude Landry, Maxime Landry",CarpeDiem Film & TV,Dawn Ford,5.40,227,"$1,875,283.00","$1,909,262.69"
277452,tt8925540,Racetime,2018,2018-12-14,Comedy,89,Canada,English,"Benoît Godbout, Jean-François Pouliot","Claude Landry, Maxime Landry",CarpeDiem Film & TV,Angela Galuppo,5.40,227,"$1,875,283.00","$1,909,262.69"
277453,tt8925540,Racetime,2018,2018-12-14,Comedy,89,Canada,English,"Benoît Godbout, Jean-François Pouliot","Claude Landry, Maxime Landry",CarpeDiem Film & TV,Holly Gauthier-Frankel,5.40,227,"$1,875,283.00","$1,909,262.69"
277454,tt8925540,Racetime,2018,2018-12-14,Comedy,89,Canada,English,"Benoît Godbout, Jean-François Pouliot","Claude Landry, Maxime Landry",CarpeDiem Film & TV,Éloïsa Laflamme-Cervantes,5.40,227,"$1,875,283.00","$1,909,262.69"


In [13]:
# Pushing the cleaned DF to a CSV file
stacked_movie_data_actors.to_csv("Data/data_cleaning_stacked_actors.csv")